In [ ]:
%load_ext blackcellmagic

#### Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from scipy.stats import ttest_rel

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES, ROI_COLORS
from spacestream.core.paths import DATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
streams = ['Ventral', 'Parietal'] #old naming

### Left panel - position

#### Load and format data

In [ ]:
results = []
missing_files = 0
for sidx, subj in enumerate(SUBJECTS):
    for seedx, seed in enumerate(seeds):
        for stidx, stream in enumerate(streams):
            for hidx, hemi in enumerate(hemis):
                fullfile = (
                    RESULTS_PATH
                    + "analyses/transfer/HVM"
                    + ("/sw0.5_seed" + str(seed))
                    + ("/subj" + subj)
                    + (
                        "/"
                        + hemi
                        + "_"
                        + stream
                        + "_stream_position_top_5k_all_var_splits_save_preds.pkl"
                    )
                )

                # load file
                if Path(fullfile).is_file():
                    o = open(fullfile, "rb")
                    metrics = pickle.load(o)

                    result = np.mean(metrics["test_acc"])
                    results.append(
                        [
                            subj,
                            seed,
                            hemi,
                            stream,
                            result,
                        ]
                    )

                else:
                    print(fullfile)
                    missing_files += 1
print(str(missing_files) + " missing files")

In [ ]:
df = pd.DataFrame(results, columns=['subject','seed', 'hemi','stream', 'result'])

#### Plotting

In [ ]:
ROI_COLORS = ['#377E2C', '#1A1AAC', '#8C1A4C']

In [ ]:
ventral = list(df[df["stream"].isin(["Ventral"])]["result"])
parietal = list(df[df["stream"].isin(["Parietal"])]["result"])

#separate hemis
ventral_left = list(df.set_index(['stream', 'hemi']).loc[("Ventral","lh")].reset_index()["result"])
parietal_left = list(df.set_index(['stream', 'hemi']).loc[("Parietal","lh")].reset_index()["result"])
ventral_right = list(df.set_index(['stream', 'hemi']).loc[("Ventral","rh")].reset_index()["result"])
parietal_right = list(df.set_index(['stream', 'hemi']).loc[("Parietal","rh")].reset_index()["result"])

fig, ax = plt.subplots(figsize=(3, 6))
# plotting the lines
for i in range(len(ventral)):
    plt.plot([0.025, 0.975], [parietal[i],ventral[i]], c="k", alpha=0.15)
# plotting the points
plt.scatter(
    np.ones(len(ventral_left)),
    ventral_left,
    color='#8C1A4C', 
    s=70,
    alpha=0.95,
    edgecolors="w",
    marker = "^",
    linewidths=0.5,
)
plt.scatter(
    np.ones(len(ventral_right)),
    ventral_right,
    color='#8C1A4C', 
    s=70,
    alpha=0.95,
    edgecolors="w",
    linewidths=0.75,
)
plt.xticks([0, 1], ["Dorsal", "Ventral"], fontsize=16)
plt.scatter(
    np.zeros(len(parietal_left)),
    parietal_left,
    color='#377E2C', 
    s=70,
    alpha=0.95,
    edgecolors="w",
    marker='^',
    linewidths=0.5,
)
plt.scatter(
    np.zeros(len(parietal_right)),
    parietal_right,
    color='#377E2C',
    s=70,
    alpha=0.95,
    edgecolors="w",
    linewidths=0.5,
)


plt.xlim([-0.2, 1.2])
plt.ylim([0.45, 0.535])

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

fig.tight_layout()
plt.show()

save_path = RESULTS_PATH + "/figures/4b_left_KGScoors.png"
fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
# reshape and then mean across hemispheres
ventral_position = np.mean(np.mean(np.reshape(ventral, ((5, 2, 8))), axis=0), axis=0)
dorsal_position = np.mean(np.mean(np.reshape(parietal, ((5, 2, 8))), axis=0), axis=0)

In [ ]:
ttest_rel(ventral_position, dorsal_position)

In [ ]:
print(np.mean(ventral_position)*100, np.std(ventral_position)*100)
print(np.mean(dorsal_position)*100, np.std(dorsal_position)*100)

### Right panel - categorization

#### Load and format data

In [ ]:
lh_results, rh_results = {"Ventral": {}, "Parietal": {}}, {
    "Ventral": {},
    "Parietal": {},
}

for stidx, stream in enumerate(streams):

    for hidx, hemi in enumerate(hemis):
        max_acc = []

        for seedx, seed in enumerate(seeds):

            for subjx, subject in enumerate(SUBJECTS):

                log_file = (
                    RESULTS_PATH
                    + "analyses/transfer/linear_eval/logs/"
                    + "spacetorch_sw0.5_"
                    + hemi
                    + "_subj"
                    + subject
                    + "_"
                    + stream
                    + "_seed"
                    + str(seed)
                    + "_training.log"
                )

                with open(log_file, "r") as f:
                    for line in f:
                        if "Max Accuracy" in line:
                            max_acc.append(float(line[-7:-2]))
        if hemi == "lh":
            lh_results[stream] = max_acc
        else:
            rh_results[stream] = max_acc

In [ ]:
ventral = np.concatenate((lh_results["Ventral"], rh_results["Ventral"]))
parietal = np.concatenate((lh_results["Parietal"], rh_results["Parietal"]))

#### Plotting

In [ ]:
sns.set_theme(style="ticks")

task = "imagenet"

fig, ax = plt.subplots(figsize=(3, 6))

# plotting the lines
for i in range(len(ventral)):
    plt.plot([0.025, 0.975], [parietal[i], ventral[i]], c="k", alpha=0.15)
# plotting the points
plt.scatter(
    np.ones(len(lh_results["Ventral"])),
    lh_results["Ventral"],
    color='#8C1A4C',
    s=70,
    alpha=0.95,
    edgecolors="w",
    marker="^",
    linewidths=0.5,
)
plt.scatter(
    np.ones(len(rh_results["Ventral"])),
    rh_results["Ventral"],
    color='#8C1A4C',
    s=70,
    alpha=0.95,
    edgecolors="w",
    linewidths=0.75,
)
plt.xticks([0, 1], ["Dorsal", "Ventral"], fontsize=16)
plt.scatter(
    np.zeros(len(lh_results["Parietal"])),
    lh_results["Parietal"],
    color='#377E2C',
    s=70,
    alpha=0.95,
    edgecolors="w",
    marker="^",
    linewidths=0.5,
)
plt.scatter(
    np.zeros(len(rh_results["Parietal"])),
    rh_results["Parietal"],
    color='#377E2C',
    s=70,
    alpha=0.95,
    edgecolors="w",
    linewidths=0.5,
)

plt.xticks([0, 1], ["Dorsal", "Ventral"], fontsize=16)
plt.xlim([-0.2, 1.2])

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

fig.tight_layout()
plt.show()

save_path = RESULTS_PATH + "/figures/4b_right_KGScolors.png"
fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
# reshape and then mean across hemispheres
ventral_categorization = np.mean(
    np.mean(np.reshape(ventral, ((5, 2, 8))), axis=0), axis=0
)
dorsal_categorization = np.mean(
    np.mean(np.reshape(parietal, ((5, 2, 8))), axis=0), axis=0
)

In [ ]:
ttest_rel(ventral_categorization, dorsal_categorization)

In [ ]:
print(np.mean(ventral_categorization), np.std(ventral_categorization))
print(np.mean(dorsal_categorization), np.std(dorsal_categorization))